In [1]:
import os, sys
sys.path.insert(0, '/clusterCAD')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "clusterCAD.settings")
import django
django.setup()
import pks.models
import compounddb.models

In [2]:
from rdkit import Chem as chem
from rdkit.Chem import AllChem, Draw, rdinchi
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
# save compounds in the database
caf = compounddb.models.Compound(name="Caffeine", smiles="O=C2N(c1ncn(c1C(=O)N2C)C)C")
caf.computeInchi()
caf.save()

cw = compounddb.models.Compound(name="cw", smiles="CCCC1=CC=C(C=C1)C(C)C(=O)O")
cw.computeInchi()
cw.save()

In [4]:
caf.inchiKey
# caf.delete()

'RYYVLZVUVIJVGH-UHFFFAOYSA-N'

In [5]:
# create RDKit index
from django.db import connection

with connection.cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS mols;")
    cursor.execute("select * into mols from (select \"inchiKey\", mol_from_smiles(smiles::cstring) m from compounddb_compound) tmp where m is not null;")
    cursor.execute("create index molidx on mols using gist(m);")
    cursor.execute("alter table mols add primary key (\"inchiKey\");")

In [6]:
compounds = compounddb.models.Compound.objects.raw('select \"inchiKey\" from mols where m@>\'CCCc1ccc(C(C)C(=O)O)cc1\';')
[x.name for x in compounds]

['cw']